### Imports

In [1]:
import os
os.chdir(os.getcwd() + '/..')
print(os.getcwd())

import data.data as data
import inout.importexport as imp
import numpy as np
import pandas as pd
import xgboost as xgb


/home/giovanni/Desktop/RecSys


### Create the complete dataframe

#### Read the recommendations from csv

In [15]:
raw_recs = imp.importcsv('submission/13-12-2018/gxboost25recommendations_12-17-23.csv', check_len=-1)

print(raw_recs[0:2])

[[7, 4492, 15779, 14714, 11257, 20242, 15167, 15908, 3648, 4469, 8749, 17154, 8985, 11641, 5309, 11636, 14400, 4189, 8275, 6615, 8362, 13123, 17780, 20443, 17495, 1063], [25, 4202, 699, 1900, 19134, 14338, 10438, 7077, 12768, 16869, 4899, 800, 2018, 4720, 11633, 10414, 11393, 18567, 11943, 15261, 19390, 12716, 11347, 1422, 7545, 1726]]


#### Explode each row into multiple rows (one per interaction)

In [16]:
recs_tracks = []
for rec in raw_recs:
    playlist_id = rec[0]
    for t in rec[1:]:
        recs_tracks.append([playlist_id, t])
recs_df = pd.DataFrame(recs_tracks, columns=['playlist_id','track_id'])

print(recs_df.head(50))

    playlist_id  track_id
0             7      4492
1             7     15779
2             7     14714
3             7     11257
4             7     20242
5             7     15167
6             7     15908
7             7      3648
8             7      4469
9             7      8749
10            7     17154
11            7      8985
12            7     11641
13            7      5309
14            7     11636
15            7     14400
16            7      4189
17            7      8275
18            7      6615
19            7      8362
20            7     13123
21            7     17780
22            7     20443
23            7     17495
24            7      1063
25           25      4202
26           25       699
27           25      1900
28           25     19134
29           25     14338
30           25     10438
31           25      7077
32           25     12768
33           25     16869
34           25      4899
35           25       800
36           25      2018
37          

#### Append the 'profile_length' column to the recommendation dataframe

In [17]:
target_ids = data.get_target_playlists()
targetURM = data.get_urm_train_1()[target_ids]
user_profile_lengths = np.array(targetURM.sum(axis=1)).flatten()
profile_lengths_df = pd.DataFrame({'playlist_id': target_ids, 'profile_length': user_profile_lengths})

print(profile_lengths_df.head(10))

   playlist_id  profile_length
0            7              28
1           25              13
2           29              18
3           34              24
4           50               8
5           52              16
6           60               8
7           64              12
8           77              29
9           80              18


In [18]:
# group users
recs_tracks = []
for rec in raw_recs:
    playlist_id = rec[0]
    for t in rec[1:]:
        recs_tracks.append([playlist_id, t])
recs_df = pd.DataFrame(recs_tracks, columns=['playlist_id','track_id'])

print(recs_df.head(50))

    playlist_id  track_id
0             7      4492
1             7     15779
2             7     14714
3             7     11257
4             7     20242
5             7     15167
6             7     15908
7             7      3648
8             7      4469
9             7      8749
10            7     17154
11            7      8985
12            7     11641
13            7      5309
14            7     11636
15            7     14400
16            7      4189
17            7      8275
18            7      6615
19            7      8362
20            7     13123
21            7     17780
22            7     20443
23            7     17495
24            7      1063
25           25      4202
26           25       699
27           25      1900
28           25     19134
29           25     14338
30           25     10438
31           25      7077
32           25     12768
33           25     16869
34           25      4899
35           25       800
36           25      2018
37          

In [19]:
rec_lengths_df = recs_df.merge(profile_lengths_df, on='playlist_id')
print(rec_lengths_df.head(40))

    playlist_id  track_id  profile_length
0             7      4492              28
1             7     15779              28
2             7     14714              28
3             7     11257              28
4             7     20242              28
5             7     15167              28
6             7     15908              28
7             7      3648              28
8             7      4469              28
9             7      8749              28
10            7     17154              28
11            7      8985              28
12            7     11641              28
13            7      5309              28
14            7     11636              28
15            7     14400              28
16            7      4189              28
17            7      8275              28
18            7      6615              28
19            7      8362              28
20            7     13123              28
21            7     17780              28
22            7     20443         

In [20]:
ohp = pd.get_dummies(rec_lengths_df['playlist_id'])
ohp = ohp.to_sparse(fill_value=0)
# rec_lengths_df = rec_lengths_df.drop(['playlist_id'], axis=1)

In [21]:
oht = pd.get_dummies(rec_lengths_df['track_id'])
oht = oht.to_sparse(fill_value=0)
rec_lengths_df = rec_lengths_df.drop(['track_id'], axis=1)

In [22]:
oh1 = pd.concat([rec_lengths_df, ohp, oht], axis=1)

oh1

,playlist_id,profile_length,3,6,7,17,18,19,20,25,...,20609,20611,20617,20621,20623,20624,20626,20627,20631,20632
0,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Append popularity feature

In [ ]:
df = pd.read_csv('/home/giovanni/Desktop/RecSys/raw_data/original_csv/train.csv')
df.groupby(['track_id']).size().reset_index(name='polularity')

#### Append the 'label' column 

In [23]:
urm_test = data.get_urm_test_1()
test_labels = []

last_playlist_id = -1
for idx,row in recs_df.iterrows():
    current_playlist_id = row['playlist_id']
    track_id = row['track_id']
    # cache the row of the urm test if same playlist of the previous iteration
    if not current_playlist_id == last_playlist_id:
        # tracks ids in the t row of urm test
        tracks_ids = urm_test.getrow(current_playlist_id).nonzero()[1]
        last_playlist_id = current_playlist_id
    
    test_labels.append(1 if track_id in tracks_ids else 0)

test_labels_df = pd.DataFrame({'label': test_labels})

In [11]:
print(len(test_labels), test_labels[0:30])

250000 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


In [24]:
oh1 = pd.concat([oh1, test_labels_df], axis=1)

In [25]:
oh1

,playlist_id,profile_length,3,6,7,17,18,19,20,25,...,20611,20617,20621,20623,20624,20626,20627,20631,20632,label
0,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,7,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
def func(x):
    n = x['label'].sum()
    ones = x.loc[x['label'] == 1]
    zeros = x.loc[x['label'] == 0].sample(n)
    return pd.concat([ones,zeros])

### Split into train and test dataframes

In [58]:
full = oh1.head(100).groupby(['playlist_id'], as_index=False).apply(func)

In [67]:
full.reset_index().drop(['level_0', 'level_1'], axis=1)

,playlist_id,profile_length,3,6,7,17,18,19,20,25,...,20611,20617,20621,20623,20624,20626,20627,20631,20632,label
0,7,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,7,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,7,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,7,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,7,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,25,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,25,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,25,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,25,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
import data.data as d
import random
import math
tgt = d.get_target_playlists()
train_tgt = random.sample(tgt, math.floor(len(tgt)/2))
test_tgt = list(set(tgt) - set(train_tgt))
train = full.loc[full['playlist_id'].isin(train_tgt)]
test = full.loc[full['playlist_id'].isin(test_tgt)]

In [81]:
train

playlist_id  profile_length  3  6  7  17  18  19  20  25  ...    20611  \
1 37           25              13  0  0  0   0   0   0   0   0  ...        0   
  44           25              13  0  0  0   0   0   0   0   0  ...        0   
  46           25              13  0  0  0   0   0   0   0   0  ...        0   
  30           25              13  0  0  0   0   0   0   0   0  ...        0   

      20617  20621  20623  20624  20626  20627  20631  20632  label  
1 37      0      0      0      0      0      0      0      0      1  
  44      0      0      0      0      0      0      0      0      1  
  46      0      0      0      0      0      0      0      0      0  
  30      0      0      0      0      0      0      0      0      0  

[4 rows x 19935 columns]

In [82]:
test

playlist_id  profile_length  3  6  7  17  18  19  20  25  ...    20611  \
0 2             7              28  0  0  0   0   0   0   0   0  ...        0   
  23            7              28  0  0  0   0   0   0   0   0  ...        0   
  24            7              28  0  0  0   0   0   0   0   0  ...        0   
  7             7              28  0  0  0   0   0   0   0   0  ...        0   
  3             7              28  0  0  0   0   0   0   0   0  ...        0   
  19            7              28  0  0  0   0   0   0   0   0  ...        0   
2 52           29              18  0  0  0   0   0   0   0   0  ...        0   
  54           29              18  0  0  0   0   0   0   0   0  ...        0   
  50           29              18  0  0  0   0   0   0   0   0  ...        0   
  58           29              18  0  0  0   0   0   0   0   0  ...        0   
3 75           34              24  0  0  0   0   0   0   0   0  ...        0   
  80           34              24  0  0  0   0   0   0   0   0  ...        0   
  81           34              24  0  0  0   0   0   0   0   0  ...        0   
  95           34              24  0  0  0   0   0   0   0   0  ...        0   

      20617  20621  20623  20624  20626  20627  20631  20632  label  
0 2       0      0      0      0      0      0      0      0      1  
  23      0      0      0      0      0      0      0      0      1  
  24      0      0      0      0      0      0      0      0      1  
  7       0      0      0      0      0      0      0      0      0  
  3       0      0      0      0      0      0      0      0      0  
  19      0      0      0      0      0      0      0      0      0  
2 52      0      0      0      0      0      0      0      0      1  
  54      0      0      0      0      0      0      0      0      1  
  50      0      0      0      0      0      0      0      0      0  
  58      0      0      0      0      0      0      0      0      0  
3 75      0      0      0      0      0      0      0      0      1  
  80      0      0      0      0      0      0      0      0      1  
  81      0      0      0      0      0      0      0      0      0  
  95      0      0      0      0      0      0      0      0      0  

[14 rows x 19935 columns]

### Train with XGBoost

In [ ]:
dtrain = xgb.DMatrix(oh1, [])